In [2]:
import wikiparser
from wikiparser import WikipediaArticle
%load_ext autoreload
%autoreload 2

In [13]:
cache = []
wtf = []
last_print = 0
titles = set(["A Tale of Two Cities", "Little Dorrit", "Hard Times", "Bleak House", "Oliver Twist", "David Copperfield"])
parser = wikiparser.WikipediaParser("/home/bschmidt/sources/wikipedia.bz2")
#parser = wikiparser.WikipediaParser("/Users/bschmidt/Downloads/Unconfirmed 628142.crdownload")
for i,article in enumerate(parser):
    if "gutenberg.org/ebooks" in article.text:
        cache.append(article)    
    if i - last_print >= 10000:
        print "{} matches in {} articles".format(len(cache),i)
        last_print = i


38 matches in 10000 articles
73 matches in 20000 articles
97 matches in 30000 articles
128 matches in 40000 articles
155 matches in 50000 articles
167 matches in 60000 articles
177 matches in 70000 articles
179 matches in 80000 articles
181 matches in 90000 articles
181 matches in 100000 articles
192 matches in 110000 articles
212 matches in 120000 articles
229 matches in 130000 articles
241 matches in 140000 articles
260 matches in 150000 articles
277 matches in 160000 articles
289 matches in 170000 articles
303 matches in 180000 articles
312 matches in 190000 articles
322 matches in 200000 articles
337 matches in 210000 articles
340 matches in 220000 articles
349 matches in 230000 articles
358 matches in 240000 articles
363 matches in 250000 articles
369 matches in 260000 articles
371 matches in 270000 articles
377 matches in 280000 articles
381 matches in 290000 articles
385 matches in 300000 articles
388 matches in 310000 articles
393 matches in 320000 articles
397 matches in 33000

KeyboardInterrupt: 

In [33]:
import random

a = random.sample(cache,1)[0]
print a.title
print a.text[:500]

Peninsular War
{{About|2=the 1862 American Civil War campaign|3=Peninsula Campaign}}
      <text xml:space="preserve">{{About|2=the 1862 American Civil War campaign|3=Peninsula Campaign}}
{{pp-pc1}}
{{very long|rps=102|date=February 2015}}
{{Use British English|date=August 2013}}
{{Use dmy dates|date=August 2013}}
{{Infobox military conflict
| conflict = Peninsular War
| partof = the [[Napoleonic Wars]]
| image = [[File:El dos de mayo de 1808 en Madrid rdit.jpg|300px]]
| caption = ''[[The Second of May 1808|Th


In [12]:
print article.text

{{About|the novel by Charles Dickens|the American illusionist|David Copperfield (illusionist)|other uses}}
      <text xml:space="preserve">{{About|the novel by Charles Dickens|the American illusionist|David Copperfield (illusionist)|other uses}}
{{Use dmy dates|date=September 2013}}
{{EngvarB|date=September 2013}}
{{Italic title}}
{{Infobox book
| name           = David Copperfield
| title_orig     = The Personal History, Adventures,&lt;br&gt;Experience and Observation&lt;br&gt;of David Copperfield&lt;br&gt;the Younger&lt;br&gt;of Blunderstone Rookery
| image          = Copperfield cover serial.jpg
| image_size     = 200px
| caption  = Cover, first serial edition of 1849
| author         = [[Charles Dickens]]
| illustrator    = [[Hablot Knight Browne]] ([[Phiz]])
| cover_artist   = [[Hablot Knight Browne]] ([[Phiz]])
| country        = United Kingdom
| language       = English
| series         = 
| genre          = [[Novel]]   ([[Bildungsroman]])
| publisher      = [[Bradbury &amp; Ev

In [89]:
#Commenting out so as not to accidentally run
#import cPickle as pickle
#pickle.dump(cache,open("cache.pickle","w"))

In [2]:
import cPickle as pickle
cache = pickle.load(open("cache.pickle"))

In [6]:
art = cache[10]
art.coords()

(40.26305555555555, -85.6763888888889)

In [9]:
years = range(1790,2020,10)

In [22]:
rows = []

for article in cache:
    out = {"title":article.title,"id":article.id}
    out["lat"],out["lon"] = article.coords()
    for year in years:
        out['y' + str(year)] = 0
    for year,population in article.town_pops():
        year = 'y' + str(year)
        if year in out: # Only store for census years
            out[year] = population
    rows.append(out)

BAD COORDS: ['42', '50', '', 'N', '78', '5', '', 'W', 'region:U'] for Wyoming County, New York
BAD COORDS: ['34.803', '-86.967}}.\n* [[Ric'] for Limestone County, Alabama
BAD COORDS: ['37.53139', 'N', '88.36833', 'W}}) '] for Hardin County, Illinois
BAD COORDS: ['region:US-CT', 'format=dms', 'd'] for Enfield, Connecticut
BAD COORDS: ['format=dms', 'display=inline'] for Gray, Georgia
BAD COORDS: ['region:US-OK', 'format=dms', 'd'] for Hastings, Oklahoma
BAD COORDS: ['region:US-OK_type:city', 'fo'] for Morrison, Oklahoma
BAD COORDS: ['36', '-87', 'region:US-TN', 'displ'] for Clarksville, Tennessee
BAD COORDS: ['40.991681', '0', 'region:US_typ'] for Harrington Park, New Jersey
BAD COORDS: ['region:US-OK', 'format=dms', 'd'] for Oaks, Oklahoma
BAD COORDS: ['28', '-82.3', 'region:US-FL', 'dis'] for Tampa Bay Area
BAD COORDS: ['44.92797', '-93.22311}}\n', 'ita'] for Hiawatha, Minneapolis


In [33]:
import pandas as pd
pd.DataFrame(rows).to_csv("/home/bschmidt/9943478/wiki.csv")

In [30]:
older = []
for article in cache:
    pops = article.town_pops()
    coords = article.coords()
    if len(pops) > 1:
        older.append([article.title,pops,len(article.text)])
print len(older)
print len(cache)

AttributeError: 'WikipediaArticle' object has no attribute 'coords'

Save to disk

In [26]:
total_pop = [(title,sum([p for (y,p) in pops]), pops[-2][1], length) for title,pops,length in older]
import pandas as pd
total_pop.sort(key = lambda x: x[1],reverse=True)
frame = pd.DataFrame(total_pop,columns=["title","total","last","length"])
frame.to_csv("city_pops.csv")

In [4]:
import random
import shelve

def keyer(x):
    return len(x[1])
older.sort(key = keyer,reverse=True)
[(o[0],len(o[1])) for o in older[:25]]

[('Demography of the United States', 36),
 ('United States', 34),
 ('Manhattan', 33),
 ('New York City', 30),
 ('Newtown, Connecticut', 28),
 ('Pittsburgh', 26),
 ('Philadelphia', 26),
 ('New Haven, Connecticut', 26),
 ('New Orleans', 26),
 ('Salem, Massachusetts', 26),
 ('Danbury, Connecticut', 26),
 ('Boston', 26),
 ('Cambridge, Massachusetts', 24),
 ('Waltham, Massachusetts', 24),
 ('Newton, Massachusetts', 24),
 ('Pittsfield, Massachusetts', 24),
 ('Attleboro, Massachusetts', 24),
 ('New Bedford, Massachusetts', 24),
 ('Taunton, Massachusetts', 24),
 ('Beverly, Massachusetts', 24),
 ('Gloucester, Massachusetts', 24),
 ('Haverhill, Massachusetts', 24),
 ('Lynn, Massachusetts', 24),
 ('Newburyport, Massachusetts', 24),
 ('Westfield, Massachusetts', 24)]

In [86]:
import random
import shelve

def keyer(x):
    return len(x[1])
older.sort(key = keyer,reverse=True)
[(o[0],len(o[1])) for o in older[:25]]

[('Demography of the United States', 36),
 ('Manhattan', 33),
 ('New York City', 30),
 ('Pittsburgh', 26),
 ('Philadelphia', 26),
 ('New Haven, Connecticut', 26),
 ('New Orleans', 26),
 ('Salem, Massachusetts', 26),
 ('Danbury, Connecticut', 26),
 ('Cambridge, Massachusetts', 24),
 ('Waltham, Massachusetts', 24),
 ('Newton, Massachusetts', 24),
 ('Pittsfield, Massachusetts', 24),
 ('Attleboro, Massachusetts', 24),
 ('New Bedford, Massachusetts', 24),
 ('Taunton, Massachusetts', 24),
 ('Beverly, Massachusetts', 24),
 ('Gloucester, Massachusetts', 24),
 ('Haverhill, Massachusetts', 24),
 ('Lynn, Massachusetts', 24),
 ('Newburyport, Massachusetts', 24),
 ('Westfield, Massachusetts', 24),
 ('Northampton, Massachusetts', 24),
 ('Malden, Massachusetts', 24),
 ('Marlborough, Massachusetts', 24)]

In [524]:
[(c[0],c[1]) for c in older if c[0]=="Connecticut"]

[('Connecticut',
  [(1790, 237946),
   (1800, 251002),
   (1810, 261942),
   (1820, 275248),
   (1830, 297675),
   (1840, 309978),
   (1850, 370792),
   (1860, 460147),
   (1870, 537454),
   (1880, 622700),
   (1890, 746258),
   (1900, 908420),
   (1910, 1114756),
   (1920, 1380631),
   (1930, 1606903),
   (1940, 1709242),
   (1950, 2007280),
   (1960, 2535234),
   (1970, 3031709),
   (1980, 3107576),
   (1990, 3287116),
   (2000, 3405565),
   (2010, 3574097)])]

In [498]:
[a.title for a in cache if "Brooklyn" in a.title]

['Brooklyn, Connecticut',
 'East Brooklyn, Illinois',
 'West Brooklyn, Illinois',
 'Brooklyn, Illinois',
 'Brooklyn, Indiana',
 'Brooklyn, Iowa',
 'Brooklyn Park, Maryland',
 'Brooklyn, Michigan',
 'Brooklyn Center, Minnesota',
 'Brooklyn Park, Minnesota',
 'Brooklyn Heights, Missouri',
 'Brooklyn, Ohio',
 'Brooklyn Heights, Ohio',
 'Brooklyn (village), Wisconsin']

In [73]:
foo = wikiparser.WikipediaArticle("Lenox, Massachusetts")

In [79]:
len(foo.town_pops())

NoPopulation: 

In [80]:
foo.pops()

[]

In [58]:
import regex as re
re.findall(tag,t)

['Historical populations', 'Historical populations']